In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

%matplotlib inline


In [ ]:
# load the input data
input_data = pd.read_csv('C:\\Users\\jp\\Desktop\\testData\\Advertising.csv')

In [ ]:
input_data.head()

In [ ]:
input_data.shape

In [ ]:
input_data.isnull().sum()

In [ ]:
#import numpy as np

In [ ]:
input_data.Radio.fillna(input_data.Radio.mean(), inplace=True)
input_data.Newspaper.fillna(input_data.Newspaper.mean(), inplace=True)

In [ ]:
input_data.isnull().sum()

In [ ]:
# import numpy as np

# input_data.Radio = input_data.Radio.fillna(np.mean(input_data.Radio))
# input_data.Newspaper = input_data.Newspaper.fillna(np.mean(input_data.Newspaper))

In [ ]:
input_data.head()

In [ ]:
input_data.describe()

#### Categorical Encoding

In [ ]:
#x = input_data['Prod_type'].astype("category").cat.codes

In [ ]:
# Convert categorical attribute Prod_type (ordinal) into encoding
# input_data['Prod_type'] = input_data['Prod_type'].astype("category", ['Basic','Medium','Higher' ]).cat.codes
# input_data.head()

In [ ]:
def Prod_type_incode(x):
    if x== "Basic":
        return 1
    elif x == "Medium" :
        return 2
    else:
        return 3

In [ ]:
input_data['Prod_type'] = input_data.Prod_type.apply(Prod_type_incode)

In [ ]:
input_data.head()

In [ ]:
# Convert Categorical var "Area" into one-hot coding
input_data = pd.get_dummies(input_data, columns=['Area'])
input_data.head()


#input_data = pd.get_dummies(input_data, columns=['Prod_type'])
#input_data = pd.get_dummies(input_data, columns=['Area', 'Prod_type']) 


In [ ]:
#y.head()

In [34]:
# seperate the predictors x and response y
x = input_data.drop('Sales', axis=1)
y = input_data['Sales']

x.head()

,TV,Radio,Newspaper,Prod_type,Area_rural,Area_suburban,Area_urban
0,230.1,37.8,69.2,1,0,1,0
1,44.5,39.3,45.1,3,1,0,0
2,17.2,45.9,69.3,2,1,0,0
3,151.5,41.3,58.5,2,0,1,0
4,180.8,10.8,58.4,3,0,1,0


In [35]:
#3 split the whole data into training and testing data

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [36]:
#1: Import the model class that is gong to be used - Classification Tree
from sklearn import tree

In [37]:
#2: Create the instance of the Model Estimator (instantiate the model)
rtree = tree.DecisionTreeRegressor()

In [38]:
rtree

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [39]:
#4: train the model based on training data to learn the pattern from historical data
rtree_model = rtree.fit(x_train, y_train)

In [40]:
# Predict on test data
y_test_pred = rtree_model.predict(x_test)

In [41]:
comp_df = x_test.copy()
comp_df['actual'] = y_test
comp_df['predicted'] = y_test_pred

comp_df.head(5)

,TV,Radio,Newspaper,Prod_type,Area_rural,Area_suburban,Area_urban,actual,predicted
58,210.8,49.6,37.7,3,0,0,1,23.8,21.7
40,202.5,22.3,31.6,1,0,1,0,16.6,15.9
34,95.7,1.4,7.4,2,1,0,0,9.5,10.1
102,280.2,10.1,21.4,2,1,0,0,14.8,14.7
184,253.8,21.3,30.0,3,1,0,0,17.6,16.1


In [ ]:
# comparison of predictive vs Actual in test data
comparision_df = x_test.copy()
#.DataFrame()
comparision_df['actual'] = y_test
comparision_df['predicted'] = y_test_pred
comparision_df.head(10)

## Evaluation metrics for regression problems

Evaluation metrics for classification problems, such as **accuracy**, are not useful for regression problems. 
We need evaluation metrics designed for comparing **continuous values**.

Here are three common evaluation metrics for regression problems:

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

**Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

In [ ]:
import numpy as np

In [ ]:
# Training Predictive Error - RMSE
y_train_pred = rtree_model.predict(x_train)
print(np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))  # Training Error RMSE = 0

In [ ]:
import numpy as  np
from sklearn import metrics

In [ ]:
# Testing Predictive Error - RMSE
y_test_pred = rtree_model.predict(x_test)
np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)) # Testing Error RMSE = 4.86

### Comparing Model RMSE with NUll RMSE

In [ ]:
# create a NumPy array with the same shape as y_test
y_null = np.zeros_like(y_test, dtype=float)
y_null
# # fill the array with the mean value of y_test
y_null.fill(y_test.mean())
y_null

In [ ]:
# compute null RMSE
np.sqrt(metrics.mean_squared_error(y_test, y_null)) # NULL RMSE = 9.57 

In [ ]:
feature_imp = pd.DataFrame()
feature_imp['col'] = x_train.columns
feature_imp['imp'] = rtree.feature_importances_

feature_imp

In [ ]:
for name, importance in zip(x_train.columns, rtree.feature_importances_):
    print(name,    '  : ', importance)

In [ ]:
# Depth Tuning 
rtree = tree.DecisionTreeRegressor(max_depth=3) # 2, 3, 4
rtree.fit(x_train, y_train)
y_pred = rtree.predict(x_test)
np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
from sklearn.externals.six import StringIO
import pydotplus

import graphviz
tree_data = tree.export_graphviz(rtree, out_file=None, 
                                 feature_names=x_train.columns, 
                                 filled=True,
                                 rounded=True, 
                                 special_characters=True
                                 )
graph = graphviz.Source(tree_data)
graph

### Model Tuning - GridSearch

In [ ]:
# SKLearn Ref for Decision Tree Classifier
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
# Model Tuning to improve the model predictive performance on test data
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = { "criterion"      : ['mse', 'mae'],    
               "max_features"   : [3, 4, 5 ], 
               "max_depth"      : [ 5, 6, 7],
               "splitter"       : ['best', 'random'],
               "min_samples_leaf"  : [ 2,3,4]
                }


In [ ]:
#optimized_tree = GridSearchCV(rtree_model, param_grid, scoring = 'r2', cv = 5)
optimized_tree = GridSearchCV(rtree_model, param_grid, scoring = 'neg_mean_absolute_error', cv = 5)

In [ ]:
optimized_tree.fit(x_train, y_train)

In [ ]:
optimized_tree.best_params_

In [ ]:
optimized_tree.best_estimator_

In [ ]:
optimized_tree.best_score_

In [ ]:
optimized_tree.grid_scores_
#optimized_tree.cv_results_

In [ ]:
optimized_tree.best_params_


In [ ]:
# Final tree based on best tuning parameters using grid search
rtree_final = optimized_tree.best_estimator_
rtree_final

In [ ]:
#rtree_final.fit(x_train, y_train)

In [ ]:
# RMSE
y_test_pred = rtree_final.predict(x_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))) # Final Model RMSE = 4.16

In [ ]:
# MAE
y_test_pred = rtree_final.predict(x_test)
print(metrics.mean_absolute_error(y_test, y_test_pred)) # Final Model RMSE = 4.16

#### Re-Tune the Model with identified improtant features


In [ ]:
input_data.columns

In [ ]:
# seperate the predictors x and response y
x = input_data[['TV', 'Radio','Prod_type_Basic', 'Prod_type_Higher', 'Prod_type_Medium']]
y = input_data['Sales']

In [ ]:
x.head()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
# Model Tuning to improve the model predictive performance on test data
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = { "criterion"      : ['mse', 'mae'],    
               "max_features"   : [3, 4, 5 ], 
               "max_depth"      : [ 5, 6, 7],
               "splitter"       : ['best', 'random'],
               "min_samples_leaf"  : [ 2,3,4]
                }


In [ ]:
#optimized_tree = GridSearchCV(rtree_model, param_grid, scoring = 'r2', cv = 5)
optimized_tree = GridSearchCV(rtree_model, param_grid, scoring = 'neg_mean_absolute_error', cv = 5)

In [ ]:
optimized_tree.fit(x_train, y_train)

In [ ]:
optimized_tree.best_score_

In [ ]:
optimized_tree.best_params_

In [ ]:
# Final tree based on best tuning parameters using grid search
rtree_final = optimized_tree.best_estimator_
rtree_final

In [ ]:
#rtree_final.fit(x_train, y_train)

In [ ]:
# RMSE
y_test_pred = rtree_final.predict(x_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))) # Final Model RMSE = 4.16

In [ ]:
# MAE
y_test_pred = rtree_final.predict(x_test)
print(metrics.mean_absolute_error(y_test, y_test_pred)) # Final Model RMSE = 4.16